In [1]:
import torch
from datasets import load_dataset
dataset = load_dataset('tweet_eval','irony')

from transformers import AutoTokenizer, AlbertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("textattack/albert-base-v2-imdb",use_fast=False)



In [2]:
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2862
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 784
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 955
    })
})

In [3]:
tokenizer

AlbertTokenizer(name_or_path='textattack/albert-base-v2-imdb', vocab_size=30000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '<unk>', 'sep_token': '[SEP]', 'pad_token': '<pad>', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}

In [4]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [5]:
tokenized_datasets["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2862
})

In [6]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2862
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 784
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 955
    })
})

In [7]:
from transformers import  Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score
model = AlbertForSequenceClassification.from_pretrained("textattack/albert-base-v2-imdb", num_labels=2)
training_args = TrainingArguments(
output_dir="./results",
# Directory for saving outputs
learning_rate=5e-5,
# Learning rate for optimization
per_device_train_batch_size=16,
# Batch size for training
per_device_eval_batch_size=16,
# Batch size for evaluation
num_train_epochs=3,
# Number of training epochs
weight_decay=0.01,
# Weight decay for regularization
evaluation_strategy="epoch",
save_strategy = "no"
# Evaluation is done at the end of each epoch
)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]
val_dataset = tokenized_datasets["validation"]
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=eval_dataset,
compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}
)

2024-03-09 10:24:01.902706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 10:24:01.902752: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 10:24:01.904352: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-09 10:24:03.003373: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
model

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [9]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [10]:
eval_results = trainer.evaluate(eval_dataset)
print(eval_results)

{'eval_loss': 1.3416439294815063, 'eval_accuracy': 0.45535714285714285, 'eval_runtime': 29.9122, 'eval_samples_per_second': 26.21, 'eval_steps_per_second': 1.638}


In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [10]:
from ray import tune
def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "per_device_train_batch_size": tune.choice([1,4 ,8 ]),
        "num_train_epochs": tune.choice([1,3,5]),

        "weight_decay": tune.loguniform(1e-4, 0.1)
    }

In [11]:
small_train_dataset = tokenized_datasets["train"].shard(index=1, num_shards=10) 
small_eval_dataset = tokenized_datasets["test"].shard(index=1, num_shards=10) 


In [12]:
num_labels = 2
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score
def model_init():
    return  DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=num_labels)

In [13]:




num_labels = num_labels
trainer = Trainer(

    model_init= model_init,

    args=training_args,

    train_dataset=small_train_dataset,

    eval_dataset=small_eval_dataset,

    compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}

    

  
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
best_trial = trainer.hyperparameter_search(

    direction="maximize",

    backend="ray",

    hp_space=ray_hp_space,

    n_trials=15

)

/home/ai/anaconda3/lib/python3.11/subprocess.py:1883: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
2024-03-07 21:25:09,277	INFO worker.py:1724 -- Started a local Ray instance.
2024-03-07 21:25:10,881	INFO tune.py:220 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-03-07 21:25:10,884	INFO tune.py:583 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-03-07 21:25:11 (running for 00:00:00.20)
Using FIFO scheduling algorithm.
Logical resource usage: 0/12 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_21-25-10
Number of trials: 15/15 (15 PENDING)
+------------------------+----------+-------+-----------------+--------------------+------------------------+----------------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |
|                        |          |       |                 |                    |                ch_size |                |
|------------------------+----------+-------+-----------------+--------------------+------------------------+----------------|
| _objective_d78da_00000 | PENDING  |       |     5.61152e-06 |                  5 |                      1 |    0.015703    |
| _objective_d78da_00001 | PENDING  |       |     1.57513e-05 |                  3

(_objective pid=7911) 2024-03-07 21:25:16.803698: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(_objective pid=7911) 2024-03-07 21:25:16.803751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(_objective pid=7911) 2024-03-07 21:25:16.805066: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(_objective pid=7911) 2024-03-07 21:25:17.928686: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(_objective pid=7911) Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and a

== Status ==
Current time: 2024-03-07 21:25:21 (running for 00:00:10.28)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_21-25-10
Number of trials: 15/15 (14 PENDING, 1 RUNNING)
+------------------------+----------+-----------------+-----------------+--------------------+------------------------+----------------+
| Trial name             | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |
|                        |          |                 |                 |                    |                ch_size |                |
|------------------------+----------+-----------------+-----------------+--------------------+------------------------+----------------|
| _objective_d78da_00000 | RUNNING  | 172.19.0.1:7911 |     5.61152e-06 |                  5 |                      1 |    0.015703    |
| _objective_d78d

  5%|▌         | 77/1435 [00:06<01:43, 13.15it/s]


...

== Status ==
Current time: 2024-03-07 21:42:10 (running for 00:16:59.73)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_21-25-10
Number of trials: 15/15 (1 RUNNING, 14 TERMINATED)
+------------------------+------------+------------------+-----------------+--------------------+------------------------+----------------+-------------+
| Trial name             | status     | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |   objective |
|                        |            |                  |                 |                    |                ch_size |                |             |
|------------------------+------------+------------------+-----------------+--------------------+------------------------+----------------+-------------|
| _objective_d78da_00014 | RUNNING    | 172.19.0.1:11212 |     3.5503e-05  |       

 99%|█████████▊| 213/216 [00:43<00:00,  5.15it/s]


== Status ==
Current time: 2024-03-07 21:42:15 (running for 00:17:04.77)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_21-25-10
Number of trials: 15/15 (1 RUNNING, 14 TERMINATED)
+------------------------+------------+------------------+-----------------+--------------------+------------------------+----------------+-------------+
| Trial name             | status     | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |   objective |
|                        |            |                  |                 |                    |                ch_size |                |             |
|------------------------+------------+------------------+-----------------+--------------------+------------------------+----------------+-------------|
| _objective_d78da_00014 | RUNNING    | 172.19.0.1:11212 |     3.5503e-05  |       

100%|██████████| 216/216 [00:44<00:00,  5.50it/s]
(_objective pid=11212) 
  0%|          | 0/5 [00:00<?, ?it/s]
(_objective pid=11212) 
 40%|████      | 2/5 [00:00<00:00,  7.58it/s]
(_objective pid=11212) 
 60%|██████    | 3/5 [00:00<00:00,  5.39it/s]
(_objective pid=11212) 
 80%|████████  | 4/5 [00:00<00:00,  4.69it/s]
2024-03-07 21:42:17,618	INFO tune.py:1042 -- Total run time: 1026.73 seconds (1026.69 seconds for the tuning loop).


== Status ==
Current time: 2024-03-07 21:42:17 (running for 00:17:06.69)
Using FIFO scheduling algorithm.
Logical resource usage: 1.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /home/ai/ray_results/_objective_2024-03-07_21-25-10
Number of trials: 15/15 (15 TERMINATED)
+------------------------+------------+------------------+-----------------+--------------------+------------------------+----------------+-------------+
| Trial name             | status     | loc              |   learning_rate |   num_train_epochs |   per_device_train_bat |   weight_decay |   objective |
|                        |            |                  |                 |                    |                ch_size |                |             |
|------------------------+------------+------------------+-----------------+--------------------+------------------------+----------------+-------------|
| _objective_d78da_00000 | TERMINATED | 172.19.0.1:7911  |     5.61152e-06 |                  

In [15]:
best_trial

BestRun(run_id='d78da_00001', objective=0.6582278481012658, hyperparameters={'learning_rate': 1.5751320499779757e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 3, 'weight_decay': 0.00029375384576328336}, run_summary=<ray.tune.analysis.experiment_analysis.ExperimentAnalysis object at 0x74c6d2530690>)

In [10]:

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]
val_dataset = tokenized_datasets["validation"]
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=eval_dataset,
compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}
)

In [17]:
for n, v in best_trial.hyperparameters.items():
    setattr(trainer.args, n, v)



In [18]:
trainer.args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,


In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.741511,0.552296
2,0.642800,0.634758,0.668367
3,0.463000,0.678941,0.668367


TrainOutput(global_step=1074, training_loss=0.5406307739030494, metrics={'train_runtime': 896.7936, 'train_samples_per_second': 9.574, 'train_steps_per_second': 1.198, 'total_flos': 205189101711360.0, 'train_loss': 0.5406307739030494, 'epoch': 3.0})

In [13]:
trainer.save_model('/home/ai/Downloads/results/last-albert-tweet')

In [14]:
predictions = trainer.evaluate(train_dataset)
print(predictions)

{'eval_loss': 0.2967953383922577, 'eval_accuracy': 0.8881900768693222, 'eval_runtime': 110.7946, 'eval_samples_per_second': 25.832, 'eval_steps_per_second': 1.616, 'epoch': 3.0}


In [15]:
predictions = trainer.evaluate(eval_dataset)
print(predictions)

{'eval_loss': 0.6789411306381226, 'eval_accuracy': 0.6683673469387755, 'eval_runtime': 30.2487, 'eval_samples_per_second': 25.918, 'eval_steps_per_second': 1.62, 'epoch': 3.0}


In [16]:
predictions = trainer.evaluate(val_dataset)
print(predictions)

{'eval_loss': 0.6768719553947449, 'eval_accuracy': 0.6827225130890052, 'eval_runtime': 36.7328, 'eval_samples_per_second': 25.999, 'eval_steps_per_second': 1.633, 'epoch': 3.0}


In [17]:
predictions = trainer.predict(val_dataset)

In [18]:
predictions.predictions

array([[ 1.2853796 , -0.9122284 ],
       [-0.9372214 ,  1.2442471 ],
       [-1.245657  ,  0.97064465],
       ...,
       [-0.94831693,  1.3448441 ],
       [-0.6246013 ,  0.45075762],
       [ 1.3422035 , -1.140004  ]], dtype=float32)

In [19]:
import torch 
predictions.predictions
pre = torch.argmax(torch.tensor(predictions.predictions), dim=1)

In [20]:
true_labels = val_dataset['label']
true_labels

[1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,


In [21]:

from sklearn.metrics import confusion_matrix
# Generate predictions, then create and print the confusion matrix
confusion_mat = confusion_matrix(true_labels, pre)
print(confusion_mat)

[[352 147]
 [156 300]]


In [22]:
predictions = trainer.predict(train_dataset)

In [23]:
predictions.predictions

array([[-2.3708892 ,  2.2129304 ],
       [ 0.8304143 , -0.8120435 ],
       [-0.16447178,  0.35819685],
       ...,
       [ 1.3283876 , -0.7837455 ],
       [ 1.7066714 , -1.2829821 ],
       [-0.61844623,  0.29359883]], dtype=float32)

In [24]:
import torch 
predictions.predictions
pre = torch.argmax(torch.tensor(predictions.predictions), dim=1)
pre

tensor([1, 0, 1,  ..., 0, 0, 1])

In [25]:
true_labels = train_dataset['label']
true_labels

[1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,


In [26]:

from sklearn.metrics import confusion_matrix
# Generate predictions, then create and print the confusion matrix
confusion_mat = confusion_matrix(true_labels, pre)
print(confusion_mat)

[[1281  136]
 [ 184 1261]]


In [20]:
import random
import torch
num_examples = 10
picks = []
lable = []
input = []
prediction = [] 
for i in range(num_examples):
        pick = random.randint(0, len(dataset['validation'])-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
        input.append(dataset['validation'][picks[i]]['text'])
        #input_token = tokenizer(dataset['validation'][picks[i]]['text'])
        lable.append(dataset['validation'][picks[i]]['label'])
        encoded_input = tokenizer(dataset['validation'][pick]['text'],return_tensors='pt').to(model.device)
        prediction.append(torch.argmax(trainer.model.forward(**encoded_input).logits, dim=1)[0].cpu().item())
    
    
    


In [21]:
print(input)
print(lable,"lable")
print(prediction,"prediction")



["I bet I'm the first to use that one", 'Chenua Achebe - when things fall apart', '@user @user the upper deck definitely seemed like Philly South.', '@user COD AW SERVERS #OFFLINE', 'My year is ending perfectly😂', "I don't know why it's so fun, but it is. cx I could probably think and find out, but yeah. No. xD", 'Thanks for the AWESOME support on our #instagram page. Much appreciated. #women #motivation #rolemode', 'Now i officially look single. Ha the', 'Porygon2 #are  found in the www.monstermmorpg. com #wild. bulbiest follow @user #working', '@user @user @user @user the Jets have pick 6 in the 2015 draft the irony blows my mind!  #NFLDraft']
[1, 0, 0, 0, 1, 0, 0, 1, 0, 1] lable
[0, 0, 1, 1, 1, 1, 1, 1, 0, 1] prediction
